In [1]:
# pip install shiba shiba-model evaluate datasets wandb arabert  accelerate -U nltk torchmetrics==0.3.2

In [2]:
from typing import Dict
import pandas as pd
import numpy as np
import torchmetrics
import torch
import transformers
from datasets import load_dataset, Dataset
from transformers import HfArgumentParser, Trainer, EvalPrediction

from shiba import ShibaForClassification, CodepointTokenizer
from training.helpers import DataArguments, get_base_shiba_state_dict,get_model_hyperparams, ShibaClassificationArgs, \
    ClassificationDataCollator

<h1> Choose the model

In [3]:
model_path = '../checkpoint-611960.pt'
seg_enable = True
bert_model_name = "aubmindlab/bert-base-arabertv02"
apply_farasa=False
file_save = 'Submit_64_05'

# Pre-Process data ( if needed)

<h1> Compute Metrics

In [4]:
def compute_metrics(pred: EvalPrediction) -> Dict:
    try:
        # Convert predictions and labels to PyTorch tensors
        # label_probs = torch.tensor(pred.predictions)
        label_probs, embeddings = pred.predictions
        labels = torch.tensor(pred.label_ids)
        label_probs = torch.exp(torch.tensor(label_probs))  # undo the log in log softmax, get indices
        # # Compute accuracy
        # accuracy = torchmetrics.functional.accuracy(label_probs, labels, num_classes=len(categories))

        # Compute F1 score
        f1_score = torchmetrics.functional.f1(label_probs, labels, average='macro', num_classes=len(categories))

        # Compute recall
        recall = torchmetrics.functional.recall(label_probs, labels, average='macro', num_classes=len(categories))

        # Compute precision
        precision = torchmetrics.functional.precision(label_probs, labels, average='macro', num_classes=len(categories))

        # print("label_probs : ", label_probs, " labels : ", labels)

        metrics = {
            # 'accuracy': accuracy.item(),
            'f1_score': f1_score.item(),
            'recall': recall.item(),
            'precision': precision.item()
        }

        # print("metrics : ", metrics)
        # raise NotImplementedError

        return metrics
    except:
        print("pred : ", pred)
        print("pred.predictions : ", pred.predictions)
        print("label_probs : ", label_probs)
        print("label_probs.size : ", label_probs.size())
        print("labels : ", labels)
        print("labels.size() : ", labels.size())

        raise NotImplementedError


In [5]:
transformers.logging.set_verbosity_info()
device = "cuda"
parser = HfArgumentParser((ShibaClassificationArgs, DataArguments))


df_testOrignal = pd.read_csv("data/emotion_no_labels_v1.0.tsv", sep="\t")

for prediction_label in ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']:
    df_train = pd.read_csv("data/2018-E-c-Ar-train.txt", sep="\t")
    df_dev = pd.read_csv("data/2018-E-c-Ar-dev.txt", sep="\t")
    df_test = df_testOrignal.copy()
    if seg_enable:
        from arabert.preprocess import ArabertPreprocessor

        arabert_prep = ArabertPreprocessor(model_name=bert_model_name,apply_farasa_segmentation=apply_farasa)
        # arabert_prep.preprocess()
        df_train['Tweet'] =  df_train['Tweet'].apply(arabert_prep.preprocess)
        df_dev['Tweet'] =  df_dev['Tweet'].apply(arabert_prep.preprocess)
        df_test['Tweet'] =  df_test['Tweet'].apply(arabert_prep.preprocess)

    print(prediction_label)
    def process_example(example: Dict) -> Dict:
        return {
            'input_ids': tokenizer.encode(example['Tweet'])['input_ids'][:model.config.max_length],
            'labels': id_by_category[example[prediction_label]]
        }
    def process_exampleTemp(example: Dict) -> Dict:
        return {
            'input_ids': tokenizer.encode(example['Tweet'])['input_ids'][:model.config.max_length],
            'labels': 0
        }
    categories = {idx: cat_name for idx, cat_name in enumerate(set(df_train[prediction_label]))}
    id_by_category = {val: key for key, val in categories.items()}

    print("categories : ", categories)
    print("id_by_category : ", id_by_category)
    df_train = Dataset.from_pandas(df_train)
    df_dev = Dataset.from_pandas(df_dev)
    df_test = Dataset.from_pandas(df_test)
    tokenizer = CodepointTokenizer()
    model_hyperparams = {'dropout': 0.1, 'deep_transformer_stack_layers': 12, 'local_attention_window': 128}
    print(model_hyperparams)
    model = ShibaForClassification(vocab_size=len(categories), **model_hyperparams)
    data_collator = ClassificationDataCollator()
    print('Loading and using base shiba states from', model_path)
    checkpoint_state_dict = torch.load(model_path)
    model.shiba_model.load_state_dict(get_base_shiba_state_dict(checkpoint_state_dict))
    training_args = ShibaClassificationArgs(
        per_device_eval_batch_size=64,
        per_device_train_batch_size=64,
        data_seed=42,
        seed=42,
        do_eval=True,
        do_predict=True,
        do_train=True,
        dropout=0.5,
        eval_accumulation_steps=None,
        eval_delay=0,
        eval_steps=100,
        evaluation_strategy='steps',
        gradient_accumulation_steps=1,
        num_train_epochs=10,
        output_dir="fine_result",
        prediction_loss_only=False,
        report_to=[],
        run_name="fine_result",
        save_strategy='no',
    )
    if seg_enable:
        compute_metrics_temp = None
    else:
        compute_metrics_temp = compute_metrics
    # print(all_data)
    trainer = Trainer(model=model,
                    args=training_args,
                    data_collator=data_collator,
                    train_dataset=df_train.map(process_example, remove_columns=list(df_train[0].keys())),
                    eval_dataset=df_dev.map(process_example, remove_columns=list(df_dev[0].keys())),
                    compute_metrics=compute_metrics_temp,
                    )
    training = trainer.train()
    pred = trainer.predict(df_test.map(process_exampleTemp, remove_columns=list(df_test[0].keys())))
    df_testOrignal[prediction_label] = [categories[x] for x in np.argmax(pred.predictions[0], axis=1)]


anger
categories :  {0: 0, 1: 1}
id_by_category :  {0: 0, 1: 1}
{'dropout': 0.1, 'deep_transformer_stack_layers': 12, 'local_attention_window': 128}


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Loading and using base shiba states from ../checkpoint-611960.pt


PyTorch: setting up devices


Map:   0%|          | 0/2278 [00:00<?, ? examples/s]

...

Using auto half precision backend
***** Running training *****
  Num examples = 2,278
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 360
  Number of trainable parameters = 120,767,234


Step,Training Loss,Validation Loss
100,0.242300,0.249686
200,0.114000,0.355041
300,0.049300,0.410897


***** Running Evaluation *****
  Num examples = 585
  Batch size = 64
***** Running Evaluation *****
  Num examples = 585
  Batch size = 64
***** Running Evaluation *****
  Num examples = 585
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

***** Running Prediction *****
  Num examples = 1000
  Batch size = 64


<h1> Saving

In [6]:
df_testOrignal[['ID', 'anger', 'anticipation', 'disgust', 'fear', 'joy',
       'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].to_csv(file_save+'/E_c.tsv', index=False, sep="\t")
pd.read_csv(file_save+"/E_c.tsv", sep="\t").head(3)

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,17439,1,0,0,0,0,0,0,0,1,0,0
1,10196,0,0,0,0,0,0,0,1,1,0,0
2,17470,1,0,0,0,0,0,0,0,1,0,0


In [7]:
print("done")

done
